### Regressão dos microdados da PNAD COVID

Regressão de dados em painel para estimadores de efeitos fixos.

Variável depedente: Realização de teletrabalho

Variáveis independentes: 

Variável t: maio a novembro de 2020.

Pacotes para manipulação e regressão

In [1]:
import pandas as pd
import numpy as np

Dados utilizados

In [18]:
pnad_covid = pd.read_parquet('../Dados/pnad_covid/pnad_covid_filtrada.parquet')

#### Tratamento

segundo nota técnica do IPEA

**Variáveis da PNAD Covid que serão filtradas:**

<img src="../Anexos/var_estudo.png" width="600" height="500">

Coluna de Data (Ano e mês)



In [64]:
pnad_covid['data'] = pd.to_datetime(pnad_covid['Ano']+'-'+pnad_covid['V1013'], format='%Y-%m')

Faixa etária (A002)

In [36]:
#grupos de 10 a partir do 20 e até o 80

#Grupos de 10 a partir do 20 e até o 80
pnad_covid['grupo_etario'] = pnad_covid['A002'].apply(lambda x:
                                                      '14-20' if x < 19
                                                      else '20-29' if x < 29
                                                      else '30-39' if x < 39
                                                      else '40-49' if x < 49
                                                      else '50-59' if x < 59
                                                      else '60-69' if x < 69
                                                      else '70-79' if x < 79
                                                      else '80 ou mais' if x >= 80
                                                      else x)

#Classificação da escolaridade conforme variável

pnad_covid['grupo_etario'] = pd.Categorical(pnad_covid['grupo_etario'], categories=[
    '14-20',
    '20-29'
    '30-39'
    '40-49'
    '50-59'
    '60-69'
    '70-79'
    '80 ou mais'    
], ordered=True)

Sexo (A003)

In [20]:
#mulher recebe valor 0 e homem valor 1

pnad_covid['A003'] = pnad_covid['A003'].map({'Mulher':0, 'Homem':1})

Cor-raça (A004)

In [21]:
#brancos e amarelos como 1 e demais 0

pnad_covid['A004'] = pnad_covid['A004'].apply(lambda x: 1 if x in ['Branca', 'Amarela'] else 0)

Escolaridade (A005)

In [22]:
#Classificação da escolaridade conforme variável

pnad_covid['A005'] = pd.Categorical(pnad_covid['A005'], categories=[
    'Sem instrução',
    'Fundamental incompleto',
    'Fundamental completa',
    'Médio incompleto',
    'Médio completo',
    'Superior incompleto',
    'Superior completo',
    'Pós-graduação, mestrado ou doutorado'
], ordered=True)



Formalidade (Composta por C007-Setor e C007B- formalidade)

In [ ]:
#setor privado sem carteira, setor publico não estatutario e sem carteira, sem contribuição para a previdencia = Informal

In [23]:
def formalidade(row):
    if row['C007'] == "Empregado do setor privado":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'
            
    elif row['C007'] == "nan":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'

    elif row['C007'] == "Conta própria":
        return 'Conta própria'

    elif row['C007'] == "Empregado do setor público (inclusive empresas de economia mista)":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'

    elif row['C007'] == "Policial militar ou bombeiro militar":
        return 'Militar'

    elif row['C007'] == "Trabalhador doméstico (empregado doméstico, cuidados, babá)":
        if row['C007B'] == 'Sim, tem carteira de trabalho assinada':
            return 'Formal'
        elif row['C007B'] == 'Sim, é servidor público estatutário':
            return 'Formal'
        elif row['C007B'] == 'Não':
            return 'Informal'
        else:
            return 'Não identificado'
        
    elif row['C007'] == "Empregador":
        return 'Empregador'
    elif row['C007'] == "Militar do exercito, marinha ou aeronáutica":
        return 'Militar'
    
    elif row['C007'] == "Trabalhador familiar não remunerado em ajuda a membro do domicílio ou parente":
        return 'Trabalhador familiar'

    elif row['C007'] == "Estava fora do mercado de trabalho (fazia apenas afazeres domésticos, cuidados de pessoas ou produção para próprio consumo)":
        return 'Fora do mercado de trabalho'

In [25]:
#criar coluna de formalidade com base em duas colunas da pnad_covid

pnad_covid['formalidade'] = pnad_covid.apply(formalidade, axis=1)

Atividade (C007D)

In [37]:
#Identificadas como agricultura, comércio, setor público, serviços e industria

def setor(row):
    if row['C007D'] == 'Agricultura, pecuária, produção florestal e pesca':
        return 'Agricultura'
    elif row['C007D'] == 'Comércio no atacado e varejo;':
        return 'Comércio'
    elif row['C007D'] == 'Administração pública (governo federal, estadual e municipal)':
        return 'Setor público'
    elif row['C007D'] in ['Educação', 
                          'Construção',
                          'Saúde humana e assistência social',
                          'Serviço doméstico remunerado (será imputado da posição na ocupação)',
                          'Serviço de alimentação (bares, restaurantes, ambulantes de alimentação)',
                          'Atividades de locação de mão de obra, segurança, limpeza, paisagismo e teleatendimento',
                          'Escritórios de advocacia, engenharia, publicidade e veterinária (Atividades profissionais, científicas e técnicas)',
                          'Reparação de veículos automotores e motocicletas',
                          'Transporte de mercadorias',
                          'Cabeleireiros, tratamento de beleza e serviços pessoais',
                          'Transporte de passageiros',
                          'Informação e comunicação (jornais, rádio e televisão, telecomunicações e informática)',
                          'Bancos, atividades financeiras e de seguros',
                          'Fornecimento de eletricidade e gás, água, esgoto e coleta de lixo',
                          'Atividade artísticas, esportivas e de recreação',
                          'Atividades imobiliárias',
                          'Organizações religiosas, sindicatos e associações',
                          'Hospedagem (hotéis, pousadas etc.)',
                          'Armazenamento, correios e serviços de entregas']:
        return 'Serviços'
    elif row['C007D'] in ['Indústria da transformação (inclusive confecção e fabricação caseira)',
                            'Extração de petróleo, carvão mineral, minerais metálicos, pedra, areia, sal etc.']:
        return 'Indústria'
    elif row['C007D'] == 'Outro':
        return 'Outros'

In [38]:
pnad_covid['setor'] = pnad_covid.apply(setor, axis=1)

Ocupação (C001, C002, C003)

In [ ]:
#ocupadas 
#    ocupadas não afastadas
#    Pessoas ocupadas exercendo atividade de maneira remota
#Pessoas afastadas
#    Afastada devido distanciamento social
#    Afastada por outras razões

In [54]:
pnad_covid['C001'].value_counts()

C001
Não    1259629
Sim     925662
Name: count, dtype: int64

In [61]:
def ocupacao(row):
    if row['C001'] == 'Sim':
            return 'Ocupado não afastado'

    elif row['C001'] == 'Não':
                if row['C002'] == 'Sim':
                    if row['C003'] == 'Estava em quarentena, isolamento, distanciamento social ou férias coletivas':
                        return 'Afastado devido distanciamento social'

                    elif row['C003'] in ['Licença remunerada por motivo de saúde ou acidente da própria pessoa',
                          'Férias, folga ou jornada de trabalho variável',
                          'Afastamento do próprio negócio/empresa por motivo de gestação, saúde, acidente, etc., sem ser remunerado por instituto de previdência',
                          'Outro motivo', 
                          'Licença maternidade ou paternidade',
                          'Fatores ocasionais (mau tempo, paralisação nos serviços de transportes, etc.)',
                          'Outro tipo de licença remunerada (estudo, paternidade, casamento, licença prêmio, etc.)']:
                        return 'Afastada por outras razões'

                elif row['C002'] == 'Não':
                    return 'Ocupado não afastado'

In [ ]:
#TODO: tratamento adequado de Ocupação
#TODO: criar chave adequada para individuos no tempo

In [62]:
pnad_covid['ocupação'] = pnad_covid.apply(ocupacao, axis=1)

In [63]:
pnad_covid['ocupação'].groupby(pnad_covid['ocupação']).count()

ocupação
Afastada por outras razões                 38838
Afastado devido distanciamento social     104941
Ocupado não afastado                     2041512
Name: ocupação, dtype: int64

---

Até aqui, as seguintes variáveis foram tratadas:
- A002: faixa etária
- A003: sexo
- A004: cor
- A005: escolaridade
- C007: Formalidade
- C007D: Setor

#### Modelo

In [46]:
#Pacotes para modelagem
import  statsmodels.api as sm 
from linearmodels.panel import PanelOLS

In [47]:
# Calculating the alpha for each fund# adding the constant vector to the exogenous matrix
exog_variables = sm.add_constant(pnad_covid[['UF', 'A003', 'A004','A005','C01012','C007']])# estimating the fixed effects model
f_effects_model = PanelOLS(
    dependent = pnad_covid['teleworkable'], # endogenous variable (Y vector)
    exog = exog_variables, # exogenous variables (X matrix)
    entity_effects = True, # True -> Fixed Effects
    #check_rank = False # True -> Balanced Panel
    )

f_effects_results = f_effects_model.fit()

ValueError: Series can only be used with a 2-level MultiIndex